In [1]:
import torch
import numpy as np
import pandas as pd
import copy
import torch.nn.functional as F
import scipy.sparse as sp
from torch_geometric.nn import GCNConv,GATConv,SAGEConv
from torch_geometric.datasets import Planetoid
from torch.nn import Linear

In [2]:
# 把标签转换成onehot
def encode_onehot(labels):                                   
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int64)
    return labels_onehot

In [3]:
def normalize(mx):                                          # 归一化
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten() ## 幂
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv) # 对角化
    mx = r_mat_inv.dot(mx) # 乘积
    #mx = mx * 10000
    return mx

In [ ]:
## 获得稳定的Tissue标签对应的数字
GTEx_exp_full = pd.read_csv("/public/home/liujunwu/workdir/scripts/GNN_Reactome/GTEx_exp/GTEx_exp_down3000.csv",sep=',',header=0)


In [ ]:
print (GTEx_exp_full.iloc[0:3,0:3])
sample_full_labels = sorted(list(set(GTEx_exp_full["SMTS"].tolist())))
sample_full_labels_dict = dict(zip(sample_full_labels,range(len(sample_full_labels))))

In [ ]:
print (list(set(GTEx_exp_full["Tissue"])))
sample_full_labels = sorted(list(set(GTEx_exp_full["Tissue"].tolist())))
sample_full_labels_dict = dict(zip(sample_full_labels,range(len(sample_full_labels))))
print (sample_full_labels_dict)

In [5]:
# 读取GTEx 表达谱文件
GTEx_exp = pd.read_csv("/public/home/liujunwu/workdir/scripts/GNN_Reactome/GTEx_exp/GTEx_exp.csv.test",sep=',',header=0)
sample_labels = np.where(encode_onehot(GTEx_exp["Tissue"]))[1]
#sample_labels = encode_onehot(GTEx_exp["Tissue"])
#print (sample_labels)
# 存储标签和真实tissue关系
sample_labels_dict = dict(zip(sample_labels,GTEx_exp["Tissue"]))
print (sample_labels_dict)
print (sample_labels_dict[7])
list_labels = []
for x in list(sample_labels):
    list_labels.append([x])
sample_labels = torch.tensor(list_labels,dtype=torch.int64)
#print (sample_labels)
print (sample_labels.dtype)
print (sample_labels[0])
print (sample_labels[0].shape)
print (len(sample_labels_dict.keys()))

{13: 'Adipose Tissue', 7: 'Muscle', 4: 'Blood Vessel', 21: 'Heart', 18: 'Uterus', 14: 'Vagina', 6: 'Breast', 9: 'Skin', 20: 'Salivary Gland', 16: 'Brain', 8: 'Adrenal Gland', 11: 'Thyroid', 3: 'Lung', 10: 'Spleen', 5: 'Pancreas', 0: 'Esophagus', 15: 'Stomach', 19: 'Colon', 1: 'Small Intestine', 17: 'Prostate', 2: 'Testis', 12: 'Nerve'}
Muscle
torch.int64
tensor([13])
torch.Size([1])
22


In [20]:
reaction_nodes = pd.read_csv("/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reactome_reaction.nodes.txt",sep='\t',header=None)
print (reaction_nodes.shape)
reaction_nodes.columns = ["Reaction","Type","Gene"]
reaction_gene_list = []
for index,row in reaction_nodes.iterrows():
    if row[2] != "" :
        reaction_gene_list.extend(str(row[2]).split(","))
common_gene_list = sorted(list(set(reaction_gene_list) & set(GTEx_exp.columns)))
common_gene_list_dict = dict(zip(sorted(common_gene_list),range(len(common_gene_list))))
#存储节点和index关系
reaction_nodes_dict = dict(zip(reaction_nodes.index,reaction_nodes["Reaction"]))
reaction_nodes_rev_dict = dict(zip(reaction_nodes["Reaction"],reaction_nodes.index))
## 初始化0矩阵
zero_matrix = pd.DataFrame(np.zeros((reaction_nodes.shape[0],len(common_gene_list))),columns = common_gene_list) 
print (zero_matrix.shape)
print (reaction_nodes.iloc[0:3,0:3])
print (zero_matrix.iloc[0:3,0:3])

(13590, 3)
(13590, 7979)
        Reaction             Type  Gene
0  R-HSA-5336466  ProteinReaction  MRS2
1  R-HSA-5216072     EwasReaction   NaN
2  R-HSA-5226964  ProteinReaction  ANKH
   A1CF  A2M  A4GNT
0   0.0  0.0    0.0
1   0.0  0.0    0.0
2   0.0  0.0    0.0


In [21]:
# 精简 GTEx_exp 的维度
GTEx_exp_match = GTEx_exp.loc[:,list(common_gene_list)]
print (GTEx_exp_match.shape)

(50, 7979)


In [22]:
print ("SLC66A1" in GTEx_exp_match.columns)

False


In [65]:
edges_file = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reactome_reaction.edges.uniq.txt"
edges_pd = pd.read_csv(edges_file,header=None,sep='\t')
#edges_pd.columns = ["Edge1","Edge2","Type","TopLevel"]
edges_pd.columns = ["Edge1","Edge2"]
edges_pd["Edge1_index"] = edges_pd["Edge1"].apply(lambda x : reaction_nodes_rev_dict[x])
edges_pd["Edge2_index"] = edges_pd["Edge2"].apply(lambda x : reaction_nodes_rev_dict[x])
#edges_trans.loc[df["Edge2"]] = edges_trans["Edge2"].apply(lambda x : reaction_nodes_dict[x])
print (edges_pd[["Edge1_index","Edge2_index"]])
edges_trans = edges_pd[["Edge1_index","Edge2_index"]]
#print (edges_trans[0:3,0:1])
#edges_trans = torch.tensor(np.ndarray(edges_trans).todense(), dtype=torch.int64)
#print (edges_trans)
edges = []
for i in range(edges_trans.shape[0]):
    edges.append([edges_trans.at[i,"Edge1_index"],edges_trans.at[i,"Edge2_index"]])
edges = torch.tensor(edges,dtype = torch.int64).T
print (edges)

      Edge1_index  Edge2_index
0           13083        13084
1           13098        13099
2           11061        11062
3           11063        11062
4           11066        11067
...           ...          ...
8508         6791         5715
8509         6792         4907
8510         6789         6790
8511        13068        13069
8512         3702         3703

[8513 rows x 2 columns]
tensor([[13083, 13098, 11061,  ...,  6789, 13068,  3702],
        [13084, 13099, 11062,  ...,  6790, 13069,  3703]])


In [28]:
from sys import getsizeof as getsize

In [40]:
not_single_nodes = reaction_nodes.query('Type == "ProteinReaction" | Type == "ComplexReaction"')
print (single_nodes.iloc[0:3,0:3])


        Reaction             Type   Gene
0  R-HSA-5336466  ProteinReaction   MRS2
2  R-HSA-5226964  ProteinReaction   ANKH
4  R-HSA-5339528  ProteinReaction  TUSC3


In [66]:
import datetime
from torch_geometric.data import Data
print (zero_matrix.shape)
print (GTEx_exp_match.shape)
gene_index_dict = dict(zip(zero_matrix.columns.to_list(),range(zero_matrix.shape[1])))
print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
GTEx_exp_ndarry =  GTEx_exp_match.values
## 每个样本的边根据入度生成小图，然后通过batch组成大图
dataset = []
for sample_index in range(GTEx_exp_match.shape[0]):
    print (sample_index)
    sample_node_feature = copy.deepcopy(zero_matrix.values)
    for reaction_index in range(reaction_nodes.shape[0]):
        if (reaction_nodes.at[reaction_index,"Type"] not in ["ProteinReaction","ComplexReaction"]):continue # 跳过非蛋白对应的reaction
        current_reaction_gene_index = [common_gene_list_dict[x] for x in reaction_nodes_dict[reaction_index] if x in common_gene_list]
        sample_node_feature[[reaction_index],[current_reaction_gene_index]] = GTEx_exp_ndarry[[sample_index],[current_reaction_gene_index]]
    sample_node_feature = Data(x=torch.tensor(sample_node_feature,dtype = torch.float32),y=sample_labels[sample_index],edge_index = edges)
    dataset.append(sample_node_feature)
    print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
    #break
print ("store all sample node features done")
print (type(dataset))
print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))

(13590, 7979)
(50, 7979)
2023-09-15  10:29:36
0
2023-09-15  10:29:47
1
2023-09-15  10:29:59
2
2023-09-15  10:30:10
3
2023-09-15  10:30:21
4
2023-09-15  10:30:32
5
2023-09-15  10:30:44
6
2023-09-15  10:30:55
7
2023-09-15  10:31:07
8
2023-09-15  10:31:18
9
2023-09-15  10:31:30
10
2023-09-15  10:31:42
11
2023-09-15  10:31:53
12
2023-09-15  10:32:05
13
2023-09-15  10:32:16
14
2023-09-15  10:32:28
15
2023-09-15  10:32:39
16
2023-09-15  10:32:50
17
2023-09-15  10:33:01
18
2023-09-15  10:33:13
19
2023-09-15  10:33:24
20
2023-09-15  10:33:35
21
2023-09-15  10:33:47
22
2023-09-15  10:33:58
23
2023-09-15  10:34:09
24
2023-09-15  10:34:20
25
2023-09-15  10:34:31
26
2023-09-15  10:34:42
27
2023-09-15  10:34:53
28
2023-09-15  10:35:04
29
2023-09-15  10:35:15
30
2023-09-15  10:35:26
31
2023-09-15  10:35:37
32
2023-09-15  10:35:49
33
2023-09-15  10:36:00
34
2023-09-15  10:36:11
35
2023-09-15  10:36:21
36
2023-09-15  10:36:32
37
2023-09-15  10:36:43
38
2023-09-15  10:36:53
39
2023-09-15  10:37:04
40
2

In [46]:
print (getsize(dataset))

472


In [ ]:
print (sp.csc_matrix(sample_node_feature))

In [ ]:
print (dataset[1])

In [ ]:
print (type(dataset[0].num_))

In [67]:
from torch_geometric.nn import GraphConv

class GNN(torch.nn.Module):
    def __init__(self,num_node_features, hidden_channels,num_classes):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        #print (hidden_channels.dtype)
        #print (num_classes.dtype)
        self.lin = Linear(hidden_channels,num_classes)

    def forward(self, x, edge_index,batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        #return F.log_softmax(x, dim=1)
        x = global_mean_pool(x, batch)
        x = F.dropout(x, p=0.5, training=self.training)
        #print (x.dtype)
        x = self.lin(x)
        return x


In [48]:
model = GNN(num_node_features=7979,hidden_channels=64,num_classes=len(sample_labels_dict.keys()))
print(model)
#print (num_classes.dtype)

GNN(
  (conv1): GraphConv(7979, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 64)
  (lin): Linear(in_features=64, out_features=22, bias=True)
)


In [49]:
print (dataset[0].x.dtype)
print (dataset[0].y.dtype)
print (dataset[0].edge_index.dtype)

torch.float32
torch.int64
torch.int64


In [ ]:
del GTEx_exp
del zero_matrix


In [52]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()
torch.set_num_threads(48)
from torch_geometric.nn import global_mean_pool
from torch_geometric.loader import DataLoader
train_dataset = dataset[0:40]
train_loader = DataLoader(train_dataset, batch_size=1,shuffle=True)

def train():
    model.train()
    for data in dataset:
        #print (data.y)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index,data.batch)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
def test(loader):
    model.eval()
    correct = 0
    for data in loader:                            # 批遍历测试集数据集。
        out = model(data.x, data.edge_index, data.batch) # 一次前向传播
        pred = out.argmax(dim=1)                         # 使用概率最高的类别
        correct += int((pred == data.y).sum())           # 检查真实标签
    return correct / len(loader.dataset)

model.train()
for epoch in range(1, 201):
    train()
    train_acc = test(train_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')

Epoch: 001, Train Acc: 0.0750
Epoch: 002, Train Acc: 0.1250
Epoch: 003, Train Acc: 0.1250
Epoch: 004, Train Acc: 0.0750
Epoch: 005, Train Acc: 0.0750
Epoch: 006, Train Acc: 0.0750
Epoch: 007, Train Acc: 0.0750
Epoch: 008, Train Acc: 0.0750
Epoch: 009, Train Acc: 0.1250
Epoch: 010, Train Acc: 0.1250
Epoch: 011, Train Acc: 0.0750
Epoch: 012, Train Acc: 0.0750
Epoch: 013, Train Acc: 0.0750
Epoch: 014, Train Acc: 0.0750
Epoch: 015, Train Acc: 0.1250
Epoch: 016, Train Acc: 0.0750
Epoch: 017, Train Acc: 0.0750
Epoch: 018, Train Acc: 0.0750
Epoch: 019, Train Acc: 0.0750
Epoch: 020, Train Acc: 0.0750
Epoch: 021, Train Acc: 0.0750
Epoch: 022, Train Acc: 0.0750
Epoch: 023, Train Acc: 0.1250
Epoch: 024, Train Acc: 0.0750
Epoch: 025, Train Acc: 0.0750
Epoch: 026, Train Acc: 0.1250
Epoch: 027, Train Acc: 0.0750
Epoch: 028, Train Acc: 0.0750
Epoch: 029, Train Acc: 0.0750
Epoch: 030, Train Acc: 0.0750
Epoch: 031, Train Acc: 0.1250
Epoch: 032, Train Acc: 0.1250
Epoch: 033, Train Acc: 0.0750


KeyboardInterrupt: 

In [ ]:
torch.save(dataset,"/public/home/liujunwu/workdir/scripts/GNN_Reactome/GTEx_exp/Torch_split/test.ptch")

In [53]:
print (dataset[0])

Data(x=[13590, 7979], edge_index=[2, 39893], y=[1])


In [54]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
 
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add') 
        # "Add" aggregation (Step 5).
        #GCN类从MessagePssing中继承得到的聚合方式：“add”
        self.lin = torch.nn.Linear(in_channels, out_channels)
 
    def forward(self, x, edge_index):
        # x has shape [N, in_channels] ——N个点，每个点in_channels维属性
        # edge_index has shape [2, E]——E条边，每条边有出边和入边
 
        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        #添加自环
 
        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)
        #对X进行线性变化
 
        # Step 3: Compute normalization.
        row, col = edge_index
        #出边和入边
        deg = degree(col, x.size(0), dtype=x.dtype)
        #各个点的入度（无向图，所以入读和出度相同）
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        #1/sqrt(di) *1/sqrt(dj)
 
        # Step 4-5: Start propagating messages.
        return self.propagate(edge_index, x=x, norm=norm)
        #进行propagate
        #propagate的内部会调用message(),aggregate()和update()
        #作为消息传播的附加参数，我们传递节点嵌入x和标准化系数norm。    
 
    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]
        #我们需要对相邻节点特征x_j进行norm标准化
        #这里x_j为一个张量，其中包含每条边的源节点特征，即每个节点的邻居。
 
        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j
        #1/sqrt(di) *1/sqrt(dj) *X_j

In [63]:
conv = GCNConv(7979, 64)
x = conv(dataset[0].x,edges )
print (edges.shape)
print (dataset[0].x.shape)
print (x.shape)

torch.Size([2, 39893])
torch.Size([13590, 7979])
torch.Size([13590, 64])


In [ ]:
from torch_geometric.data import Dataset, download_url


class MyOwnDataset(Dataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data_1.pt', 'data_2.pt', ...]

    #def download(self):
        # Download to `self.raw_dir`.
     #   path = download_url(url, self.raw_dir)
     #   ...

    def process(self):
        idx = 0
        for raw_path in self.raw_paths:
            # Read data from `raw_path`.
            data = Data(...)

            if self.pre_filter is not None and not self.pre_filter(data):
                continue

            if self.pre_transform is not None:
                data = self.pre_transform(data)

            torch.save(data, osp.join(self.processed_dir, f'data_{idx}.pt'))
            idx += 1

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data

In [ ]:
import os
print (os.getcwd())
print(model)
print (globals().keys())

In [ ]:
#删除clear_env.py
def __clear_env():
    for key in globals().keys():
        if not key.startswith("__"):# 排除系统内建函数
            globals().pop(key)
_clear_env()

In [ ]:
from torch.utils.data import Dataset

In [ ]:
class Mydataset(Dataset):
    def __init__(self,):
        self.datas=[1,2,(4,'a')]
    
    def __len__(self,):
        return len(self.datas)
    
    def __getitem__(self,index):
        
        return self.datas[index]

In [ ]:
mydataset = Mydataset()

In [ ]:
mydataset[0]

In [ ]:
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
print (len(dataset))
print (os.getcwd())
loader = DataLoader(dataset[0:10],batch_size=32,shuffle=True)

In [ ]:
print (len(loader))

In [ ]:
sample_node_feature = Data(x=torch.tensor([sample_node_feature,dtype = torch.float32),y=sample_labels[sample_index],edge_index = edges)

In [ ]:
from torch_geometric.data import Data
from torch_geometric.data.batch import Batch
edge_index_s = torch.tensor([
    [0, 0, 0, 0],
    [1, 2, 3, 4],
])
x_s = torch.randn(5, 8864)  # 5 nodes.
edge_index_t = torch.tensor([
    [0, 0, 0],
    [1, 2, 3],
])
x_t = torch.randn(4, 8864)  # 4 nodes.

edge_index_3 = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x_3 = torch.randn(4, 8864)

data1= Data(x=x_s,edge_index=edge_index_s)
data2= Data(x=x_t,edge_index=edge_index_t)
data3= Data(x=x_3,edge_index=edge_index_3)
data4= Data(x=x_s,edge_index=edge_index_3)
#上面是构建3张Data图对象
# * `Batch(Data)` in case `Data` objects are batched together
#* `Batch(HeteroData)` in case `HeteroData` objects are batched together

data_list = [data1, data2,data3,data4]
loader = Batch.from_data_list(data_list)#调用该函数data_list里的data1、data2、data3 三张图形成一张大图，也就是batch
loader2 = Batch.from_data_list([data1,data3])

In [ ]:
from sys import getsizeof
print (loader)
print (loader2)
print (getsizeof(loader))
print (getsizeof(data1))
getsize(data2)
getsize(data3)
getsize(data4)
